## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_API_Key

# Configure gmaps API key
gmaps.configure(api_key=google_API_Key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qaanaaq,77.4840,-69.3632,26.47,57,100,3.11,GL,2022-05-15 17:31:24
1,1,Rikitea,-23.1203,-134.9692,77.23,74,100,17.56,PF,2022-05-15 17:31:24
2,2,Hermanus,-34.4187,19.2345,61.81,72,0,9.84,ZA,2022-05-15 17:31:25
3,3,Narasimharajapura,13.6167,75.5167,71.17,95,100,6.06,IN,2022-05-15 17:31:25
4,4,Dikson,73.5069,80.5464,31.60,99,100,8.72,RU,2022-05-15 17:31:26


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Rikitea,-23.1203,-134.9692,77.23,74,100,17.56,PF,2022-05-15 17:31:24
9,9,Atuona,-9.8000,-139.0333,77.92,78,14,18.34,PF,2022-05-15 17:31:28
17,17,Albany,42.6001,-73.9662,79.63,71,78,4.83,US,2022-05-15 17:30:32
22,22,Ponta Do Sol,32.6667,-17.1000,76.71,69,51,7.63,PT,2022-05-15 17:31:34
24,24,Hilo,19.7297,-155.0900,75.27,91,100,9.22,US,2022-05-15 17:30:25
30,30,Marzuq,14.4000,46.4667,77.29,16,1,2.51,YE,2022-05-15 17:31:37
34,34,Bubaque,11.2833,-15.8333,79.83,80,81,19.71,GW,2022-05-15 17:31:39
39,39,Bethel,41.3712,-73.4140,79.05,71,75,9.22,US,2022-05-15 17:31:42
43,43,Tautira,-17.7333,-149.1500,79.74,77,26,14.67,PF,2022-05-15 17:31:44
53,53,Kapaa,22.0752,-159.3190,75.52,91,75,13.80,US,2022-05-15 17:32:48


In [30]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0    0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       1
Date          0
dtype: int64

In [31]:
preferred_cities_df.notnull().sum()

Unnamed: 0    87
City          87
Lat           87
Lng           87
Max Temp      87
Humidity      87
Cloudiness    87
Wind Speed    87
Country       86
Date          87
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,77.23,-23.1203,-134.9692,
9,Atuona,PF,77.92,-9.8000,-139.0333,
17,Albany,US,79.63,42.6001,-73.9662,
22,Ponta Do Sol,PT,76.71,32.6667,-17.1000,
24,Hilo,US,75.27,19.7297,-155.0900,
30,Marzuq,YE,77.29,14.4000,46.4667,
34,Bubaque,GW,79.83,11.2833,-15.8333,
39,Bethel,US,79.05,41.3712,-73.4140,
43,Tautira,PF,79.74,-17.7333,-149.1500,
53,Kapaa,US,75.52,22.0752,-159.3190,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_API_Key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
#Hotel_df_clean=preferred_cities_df.dropna()
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,77.23,-23.1203,-134.9692,People ThankYou
9,Atuona,PF,77.92,-9.8000,-139.0333,Villa Enata
17,Albany,US,79.63,42.6001,-73.9662,
22,Ponta Do Sol,PT,76.71,32.6667,-17.1000,Hotel do Campo
24,Hilo,US,75.27,19.7297,-155.0900,Hilo Hawaiian Hotel


In [41]:
# 8a. Create the output File (CSV)

output_data_file = "/Users/adriennezane/Desktop/Data Bootcamp/World_Weather_Analysis/weather_data/Vacation_Search/WeatherPy_vacation.csv"



 
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
Marker=hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig=gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))